# Import library

In [67]:
import os
import sys

# add current directory to system path
sys.path.insert(0, os.getcwd())
from pathlib import Path

from IPython.display import display
import numpy as np
import pandas as pd


from pyDLCbehavior import glob_files, NovelObjectRecognitionAnalysis


# Setting the data directory

In [133]:
HOMEDIR = "NOR"
files_dict = glob_files(HOMEDIR)

csv_list = files_dict.get(".csv")
pkl_list = files_dict.get(".pickle")
video_list = files_dict.get(".avi")


[PosixPath('NOR/00301_subset1DLC_resnet50_Colab_200204Feb4shuffle1_781000.csv'), PosixPath('NOR/00301_subset2DLC_resnet50_Colab_200204Feb4shuffle1_781000.csv')]
[PosixPath('NOR/00301_subset2DLC_resnet50_Colab_200204Feb4shuffle1_781000includingmetadata.pickle'), PosixPath('NOR/00301_subset1DLC_resnet50_Colab_200204Feb4shuffle1_781000includingmetadata.pickle')]
None


# Run

In [ ]:
# example workflow
def workflow(nor: NovelObjectRecognitionAnalysis) -> None:
    # select object
    nor.select_objects(num_of_objects=2)

    # [OPTION] add object manually
    # nor.add_object(Roi(x = 200, y = 50))
    # nor.add_object(Roi(x = 50, y = 300))
    nor.analyze()

    nor.plot_trajectory()
    nor.plot_filter_scatter()
    nor.plot_heatmap()

    nor.to_pickle()


nor_list = []
for csv, pkl, video in zip(csv_list, pkl_list, video_list):
    nor = NovelObjectRecognitionAnalysis(csv, pkl, video)
    workflow(nor)
    nor_list.append(nor)


# Pd DataFrame

In [ ]:

discriminate_df = pd.concat([n.summary_df for n in nor_list], axis = 0)
discriminate_df.filename = discriminate_df.filename.apply(lambda x: x.split("DLC")[0])
display(discriminate_df)


# Total distance and Speed 

In [ ]:
linkpath = Path(r'D:\Behavior\Exp1\Exp1.R1\Data\Exp1_1_2020Jan_NOR\200211_Exp_1_1_2_NOR\Test2h\Exp1.R1_1_2nd_NOR_id_food.csv')
link_df = pd.read_csv(linkpath, header = 0)
total = pd.merge(link_df, discriminate_df, on = "filename")
total

#  DI column which depends on Zone

In [ ]:
total['DI']= total['Discrimination_index_Zone1_to_Zone2'].where(total['novel'] == 'Zone1', total['Discrimination_index_Zone2_to_Zone1'])
